In [1]:
import pandas as pd
import json
import os

# Read all CSV files
df1 = pd.read_csv('/home/srawash@iit.local/ippocrate_repo_edo/federated_models/genomic_regressor/data/client1_data_val.csv')
df2 = pd.read_csv('/home/srawash@iit.local/ippocrate_repo_edo/federated_models/genomic_regressor/data/client1_data.csv')
df3 = pd.read_csv('/home/srawash@iit.local/ippocrate_repo_edo/federated_models/genomic_regressor/data/client2_data_val.csv')
df4 = pd.read_csv('/home/srawash@iit.local/ippocrate_repo_edo/federated_models/genomic_regressor/data/client2_data.csv')

# Concatenate vertically
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Change pseudo_id to sample1, sample2, ...
df['pseudo_id'] = [f'sample{i+1}' for i in range(len(df))]

# Change label: 0 -> "disease_1", 1 -> "disease_2"
df['label'] = df['label'].map({0: 'disease_1', 1: 'disease_2'})

# Create output folder
output_dir = '/home/srawash@iit.local/ippocrate_repo_edo/federated_models/genomic_regressor/data/input_genomics_test'
os.makedirs(output_dir, exist_ok=True)

# Save JSON files for each row (excluding pseudo_id and label)
data_columns = [col for col in df.columns if col not in ['pseudo_id', 'label']]
for idx, row in df.iterrows():
    sample_name = row['pseudo_id']
    row_data = row[data_columns].to_dict()
    json_path = os.path.join(output_dir, f'{sample_name}_vcf2matrix.json')
    with open(json_path, 'w') as f:
        json.dump(row_data, f, indent=2)

# Generate mapping CSV with columns: patient_id, sample_id, sample_date, batch_id, label
mapping_df = pd.DataFrame({
    'patient_id': [f'patient{i+1}' for i in range(len(df))],
    'sample_id': df['pseudo_id'],
    'sample_date': '',
    'batch_id': '',
    'label': df['label']
})

mapping_csv_path = os.path.join(output_dir, 'mapping.csv')
mapping_df.to_csv(mapping_csv_path, index=False)

print(f"Saved {len(df)} JSON files and mapping.csv to {output_dir}")

Saved 100 JSON files and mapping.csv to /home/srawash@iit.local/ippocrate_repo_edo/federated_models/genomic_regressor/data/input_genomics_test
